In [5]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer
import pandas as pd
from datasets import Dataset
import re
from sentence_transformers.losses import TripletLoss
from sentence_transformers.training_args import SentenceTransformerTrainingArguments, BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator
import tensorflow as tf
import torch
from sklearn.metrics.pairwise import cosine_similarity

2025-04-18 10:46:21.096853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744973181.564675      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744973181.683952      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
model = SentenceTransformer('all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [64]:
model2 = SentenceTransformer('all-MiniLM-L12-v2')

In [60]:
embedding = model.encode(["Kubernetes", "Docker"])
similarities = cosine_similarity([embedding[0]], [embedding[1]])[0]
print(similarities)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0.6090292]


In [9]:
df_original = pd.read_json("/kaggle/input/rjdb-csv-format/train.jsonl",lines=True)
df_preprocessed_train = pd.read_json("/kaggle/input/rjdb-csv-format/train.jsonl",lines=True)
df_preprocessed_val = pd.read_csv("/kaggle/input/rjdb-csv-format/dev.csv")
df_preprocessed_test = pd.read_csv("/kaggle/input/rjdb-csv-format/test.csv")

df_original = df_original.iloc[:, 0:3]
df_preprocessed_train = df_preprocessed_train.iloc[:, 0:3]
df_preprocessed_val = df_preprocessed_val.iloc[:, 0:3]
df_preprocessed_test = df_preprocessed_test.iloc[:, 0:3]

In [10]:
print(df_preprocessed_train.iloc[0,0])



## Job Title
Oracle SCM Developer

## Job Summary
The role of Oracle SCM Developer calls for an experienced professional with expertise in using SQL, the ERP tool, and thorough understanding of Supply Chain Management, Bills of Materials (BOM), and Work in Progress (WIP) sections. We are looking for an outright professional who has been pivotal in handling comprehensive SCM operations, developing Oracle-run resourcing frameworks, analyzing scenario-driven networks, and facilitating cloud-based endeavors towards the resultant development processes.

## Required Skills
- Proficiency in SQL for creating computing solutions.
- In-depth knowledge of ERP to integrate various organizational systems.
- Thorough understanding of Supply chain operations including cost curation & evaluations.  
- Comprehensive use of BOM systems.
- Demonstrative experience with Work in Progress (WIP) sections.

## Required Experience
- 3 years as an Oracle SCM Developer.
- 1 year as a SAP Sales and Supply Chain

In [11]:
df_preprocessed_train.head()

,Job-Description,Resume-matched,Resume-unmatched
0,\n\n## Job Title\nOracle SCM Developer\n\n## J...,\n\n## Personal Information\nName: Konstantina...,\n\n## Personal Information\nName: Konstantina...
1,"\n\n## Job Title\nElectrical, Electronic, and ...",\n\n## Personal Information\nName: Yelena Belo...,\n\n## Personal Information\nName: Yelena Belo...
2,\n\n## Job Title\nUI - Sr. Java Tech Lead (Por...,\n\n## Personal Information\nName: Matthew Ste...,\n\n## Personal Information\nName: Matthew Jam...
3,\n\n## Job title\nTechnical Architect- JAVA\n\...,\n\n## Personal Information\nName: Haruna Yaku...,\n\n## Personal Information\nName: Haruna Yaku...
4,\n\n## Job title\nFishing and Hunting Worker\n...,\n\n## Personal Information\nName: Finley Murr...,\n\n## Personal Information\nName: Finley Murr...


In [12]:
def clean_text(text):
    text = re.sub(r'#*', '', text)
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [13]:
df_preprocessed_train["Job-Description"] = df_preprocessed_train["Job-Description"].apply(clean_text)
df_preprocessed_train["Resume-matched"] = df_preprocessed_train["Resume-matched"].apply(clean_text)
df_preprocessed_train["Resume-unmatched"] = df_preprocessed_train["Resume-unmatched"].apply(clean_text)

df_preprocessed_val["Job-Description"] = df_preprocessed_val["Job-Description"].apply(clean_text)
df_preprocessed_val["Resume-matched"] = df_preprocessed_val["Resume-matched"].apply(clean_text)
df_preprocessed_val["Resume-unmatched"] = df_preprocessed_val["Resume-unmatched"].apply(clean_text)

df_preprocessed_test["Job-Description"] = df_preprocessed_test["Job-Description"].apply(clean_text)
df_preprocessed_test["Resume-matched"] = df_preprocessed_test["Resume-matched"].apply(clean_text)
df_preprocessed_test["Resume-unmatched"] = df_preprocessed_test["Resume-unmatched"].apply(clean_text)

In [14]:
df_preprocessed_train.head()

,Job-Description,Resume-matched,Resume-unmatched
0,Job Title Oracle SCM Developer Job Summary The...,Personal Information Name: Konstantina Caruso ...,Personal Information Name: Konstantina Caruso ...
1,"Job Title Electrical, Electronic, and Electrom...",Personal Information Name: Yelena Belova Email...,Personal Information Name: Yelena Belova Email...
2,Job Title UI - Sr. Java Tech Lead (Portals) & ...,Personal Information Name: Matthew Stevens Ema...,Personal Information Name: Matthew James E-mai...
3,Job title Technical Architect- JAVA Job Summar...,Personal Information Name: Haruna Yakubu Email...,Personal Information Name: Haruna Yakubu Email...
4,Job title Fishing and Hunting Worker Job Summa...,Personal Information Name: Finley Murrin Email...,Personal Information Name: Finley Murrin Email...


In [15]:
df_original.iloc[200,0]

'\n\n## Job title\nFinancial Specialist - All Other\n\n## Job Summary\nThe financial specialist is assume high-level responsibility in the fields of project management, developing, organizing, and executing DNS operations while displaying robust PowerShell script knowledge. Diligently maintaining the Active Directory and utilizing MSCE knowledge for consistent development of technically sound practices. \n\n## Required Skills\n- Director\n- DNS\n- Management Skills with major proficiency in Team handling and budgeting.\n- MCSE\n- PowerShell\n- Exceptional skillsets in handling Project and Project Management including mitigating risk factors, working on financial projections, strategy making, allocation of resources, and tracking project accuracy.\n- Scripting capabilities \n\n## Required Experience\n- Prior experience of 2 years as a Financial specialist in a challenging environment. \n- Provision of 5 years as an Active Directory Systems Engineer exhibiting TCP/IP, DNS, scripting.\n\n

In [16]:
df_preprocessed_train.iloc[200,0]

'Job title Financial Specialist - All Other Job Summary The financial specialist is assume high-level responsibility in the fields of project management, developing, organizing, and executing DNS operations while displaying robust PowerShell script knowledge. Diligently maintaining the Active Directory and utilizing MSCE knowledge for consistent development of technically sound practices. Required Skills - Director - DNS - Management Skills with major proficiency in Team handling and budgeting. - MCSE - PowerShell - Exceptional skillsets in handling Project and Project Management including mitigating risk factors, working on financial projections, strategy making, allocation of resources, and tracking project accuracy. - Scripting capabilities Required Experience - Prior experience of 2 years as a Financial specialist in a challenging environment. - Provision of 5 years as an Active Directory Systems Engineer exhibiting TCP/IP, DNS, scripting. Responsibilities - Overseeing financial de

In [17]:
anchors = [i.iloc[0] for n,i in df_preprocessed_train.iterrows()]
positives = [i.iloc[1] for n,i in df_preprocessed_train.iterrows()]
negatives = [i.iloc[2] for n,i in df_preprocessed_train.iterrows()]

anchors_val = [i.iloc[0] for n,i in df_preprocessed_val.iterrows()]
positives_val = [i.iloc[1] for n,i in df_preprocessed_val.iterrows()]
negatives_val = [i.iloc[2] for n,i in df_preprocessed_val.iterrows()]

anchors_test = [i.iloc[0] for n,i in df_preprocessed_test.iterrows()]
positives_test = [i.iloc[1] for n,i in df_preprocessed_test.iterrows()]
negatives_test = [i.iloc[2] for n,i in df_preprocessed_test.iterrows()]

In [18]:
train_dataset = Dataset.from_dict({
    "anchor": anchors,
    "positive": positives,
    "negative": negatives
})

val_dataset = Dataset.from_dict({
    "anchor": anchors_val,
    "positive": positives_val,
    "negative": negatives_val
})

test_dataset = Dataset.from_dict({
    "anchor": anchors_test,
    "positive": positives_test,
    "negative": negatives_test
})

In [19]:
loss = TripletLoss(model)

In [20]:
loss2 = TripletLoss(model2)

In [21]:
args = SentenceTransformerTrainingArguments(
    output_dir="/kaggle/working/",

    num_train_epochs=3,                      
    per_device_train_batch_size=16,          
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=1,           

    warmup_ratio=0.1,
    learning_rate=2e-5,                      

    batch_sampler=BatchSamplers.NO_DUPLICATES,

    eval_strategy="steps",
    eval_steps=500,                          
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,                      

    logging_steps=50,                        
    logging_dir="/kaggle/working/logs",      
    report_to=[],                            

    seed=42,
    run_name="mpnet-triplet-50k"
)

In [22]:
test_evaluator = TripletEvaluator(
    anchors=test_dataset["anchor"],
    positives=test_dataset["positive"],
    negatives=test_dataset["negative"],
    name="Evaluation mpnet-v2",
)
test_evaluator(model)

{'Evaluation mpnet-v2_cosine_accuracy': 0.7350000143051147}

In [23]:
test_evaluator = TripletEvaluator(
    anchors=test_dataset["anchor"],
    positives=test_dataset["positive"],
    negatives=test_dataset["negative"],
    name="Evaluation mpnet-v2",
)
test_evaluator(model2)

{'Evaluation mpnet-v2_cosine_accuracy': 0.7099999785423279}

In [24]:
# Access the underlying transformer model
transformer = model._first_module().auto_model

# Freeze all layers
for param in transformer.parameters():
    param.requires_grad = False

# Unfreeze last N transformer layers (e.g., last 2)
unfreeze_layers = 4
for i in range(1, unfreeze_layers + 1):
    for param in transformer.encoder.layer[-i].parameters():
        param.requires_grad = True

In [65]:
# Access the underlying transformer model
transformer = model2._first_module().auto_model

# Freeze all layers
for param in transformer.parameters():
    param.requires_grad = False

# Unfreeze last N transformer layers (e.g., last 2)
unfreeze_layers = 1
for i in range(1, unfreeze_layers + 1):
    for param in transformer.encoder.layer[-i].parameters():
        param.requires_grad = True

In [26]:
# Access the base transformer model
transformer = model._first_module().auto_model

# Check number of encoder layers
print(f"\nTransformer has {len(transformer.encoder.layer)} encoder layers.")

# Print which layers are frozen or not
for i, layer in enumerate(transformer.encoder.layer):
    status = all(not p.requires_grad for p in layer.parameters())
    print(f"Layer {i}: {'❄️ Frozen' if status else '🔥 Trainable'}")


Transformer has 12 encoder layers.
Layer 0: ❄️ Frozen
Layer 1: ❄️ Frozen
Layer 2: ❄️ Frozen
Layer 3: ❄️ Frozen
Layer 4: ❄️ Frozen
Layer 5: ❄️ Frozen
Layer 6: ❄️ Frozen
Layer 7: ❄️ Frozen
Layer 8: 🔥 Trainable
Layer 9: 🔥 Trainable
Layer 10: 🔥 Trainable
Layer 11: 🔥 Trainable


In [66]:
# Access the base transformer model
transformer = model2._first_module().auto_model

# Check number of encoder layers
print(f"\nTransformer has {len(transformer.encoder.layer)} encoder layers.")

# Print which layers are frozen or not
for i, layer in enumerate(transformer.encoder.layer):
    status = all(not p.requires_grad for p in layer.parameters())
    print(f"Layer {i}: {'❄️ Frozen' if status else '🔥 Trainable'}")


Transformer has 12 encoder layers.
Layer 0: ❄️ Frozen
Layer 1: ❄️ Frozen
Layer 2: ❄️ Frozen
Layer 3: ❄️ Frozen
Layer 4: ❄️ Frozen
Layer 5: ❄️ Frozen
Layer 6: ❄️ Frozen
Layer 7: ❄️ Frozen
Layer 8: ❄️ Frozen
Layer 9: ❄️ Frozen
Layer 10: ❄️ Frozen
Layer 11: 🔥 Trainable


In [28]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total:,}")
    print(f"Trainable parameters: {trainable:,}")
    print(f"Frozen parameters: {total - trainable:,}")

In [29]:
count_parameters(model)

Total parameters: 109,486,464
Trainable parameters: 28,351,488
Frozen parameters: 81,134,976


In [67]:
count_parameters(model2)

Total parameters: 33,360,000
Trainable parameters: 1,774,464
Frozen parameters: 31,585,536


In [31]:
evaluator = TripletEvaluator(
    anchors=val_dataset["anchor"],
    positives=val_dataset["positive"],
    negatives=val_dataset["negative"],
    name="all-mpnet-v2",
)
evaluator(model)

{'all-mpnet-v2_cosine_accuracy': 0.7281988859176636}

In [32]:
evaluator2 = TripletEvaluator(
    anchors=val_dataset["anchor"],
    positives=val_dataset["positive"],
    negatives=val_dataset["negative"],
    name="all-MINImpnet-v2",
)
evaluator(model2)

{'all-mpnet-v2_cosine_accuracy': 0.7127139568328857}

In [33]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    loss=loss,
    evaluator=evaluator,
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,All-mpnet-v2 Cosine Accuracy
500,4.401600,4.353424,0.906275
1000,4.279500,4.537661,0.911980
1500,4.326500,4.319248,0.931133
2000,4.240600,4.281950,0.931540
2500,4.237500,4.256231,0.922575
3000,4.151400,4.215879,0.935615
3500,4.147300,4.161713,0.944173
4000,4.141200,4.148143,0.931540
4500,4.124600,4.122781,0.920945


TrainOutput(global_step=4689, training_loss=4.26062067039589, metrics={'train_runtime': 9040.8071, 'train_samples_per_second': 16.591, 'train_steps_per_second': 0.519, 'total_flos': 0.0, 'train_loss': 4.26062067039589, 'epoch': 3.0})

In [34]:
test_evaluator = TripletEvaluator(
    anchors=test_dataset["anchor"],
    positives=test_dataset["positive"],
    negatives=test_dataset["negative"],
    name="Evaluation mpnet-v2",
)
test_evaluator(model)

{'Evaluation mpnet-v2_cosine_accuracy': 0.9089999794960022}

In [61]:
args2 = SentenceTransformerTrainingArguments(
    output_dir="/kaggle/working/",

    num_train_epochs=1,                      
    per_device_train_batch_size=16,          
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=1,           

    warmup_ratio=0.1,
    learning_rate=2e-5,                      

    batch_sampler=BatchSamplers.NO_DUPLICATES,

    eval_strategy="steps",
    eval_steps=500,                          
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,                      

    logging_steps=50,                        
    logging_dir="/kaggle/working/logs",      
    report_to=[],                            

    seed=42,
    run_name="MINImpnet-triplet-50k"
)

In [68]:
trainer2 = SentenceTransformerTrainer(
    model=model2,
    args=args2,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    loss=loss2,
    evaluator=evaluator2,
)
trainer2.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,All-minimpnet-v2 Cosine Accuracy
500,4.736900,4.869995,0.712714
1000,4.413900,4.389201,0.712714
1500,4.446700,4.464349,0.712714


TrainOutput(global_step=1563, training_loss=4.642954239994764, metrics={'train_runtime': 461.7571, 'train_samples_per_second': 108.282, 'train_steps_per_second': 3.385, 'total_flos': 0.0, 'train_loss': 4.642954239994764, 'epoch': 1.0})

In [71]:
test_evaluator(model2)

{'Evaluation mpnet-v2_cosine_accuracy': 0.7099999785423279}

In [72]:
trainer.save_model("/kaggle/working/fine_tuned_mpnet")

In [73]:
trainer2.save_model("/kaggle/working/fine_tuned_MINImpnet")

In [74]:
import shutil
shutil.make_archive("/kaggle/working/fine_tuned_mpnet 5", 'zip', "/kaggle/working/fine_tuned_mpnet")

'/kaggle/working/fine_tuned_mpnet 5.zip'

In [75]:
shutil.make_archive("/kaggle/working/fine_tuned_MINImpnet 5", 'zip', "/kaggle/working/fine_tuned_MINImpnet")

'/kaggle/working/fine_tuned_MINImpnet 5.zip'